In [3]:
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, TimestampType, LongType
import json
from datetime import datetime

## Create Spark session

spark = SparkSession.builder.enableHiveSupport().config("spark.sql.legacy.createHiveTableByDefault", False).appName("UserInformation").getOrCreate()
sc = spark.sparkContext
sparksql = SQLContext(sc)

In [4]:
# Set the warehouse location
warehouse_location = "file:///home/hx152/Project/tweet_project/Dataset/spark-warehouse"
hivedb_location = "file:///home/hx152/Project/tweet_project/Dataset/metastore_db"
# warehouse_location = "hdfs://data/"

spark = SparkSession.builder \
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.warehouse.dir", warehouse_location) \
    .config("spark.sql.legacy.createHiveTableByDefault", False) \
    .appName("UserInformation").getOrCreate()
sc = spark.sparkContext
sparksql = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/22 21:29:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/hx152/anaconda3/envs/pyspark/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Create DataBase

In [5]:
a = sparksql.sql('SHOW DATABASES')
a.collect()

23/04/22 21:29:42 WARN SharedState: Not allowing to set hive.metastore.warehouse.dir in SparkSession's options, please use spark.sql.warehouse.dir to set statically for cross-session usages


[Row(namespace='default'),
 Row(namespace='user_data'),
 Row(namespace='userinformation')]

In [4]:
sql_query = 'CREATE DATABASE IF NOT EXISTS user_data'
a = sparksql.sql(sql_query)
a.collect()

[]

In [6]:
a = sparksql.sql('Use user_data')
a.collect()

[]

## Create tables in selected DataBase

In [19]:
sparksql.sql('''
            CREATE TABLE IF NOT EXISTS user_basic_info(
             tweet_create_time TIMESTAMP,
             id LONG,
             name STRING,
             screen_name STRING,
             verified BOOLEAN,
             created_at TIMESTAMP,
             followers_count INT,
             friends_count INT,
             listed_count INT,
             favourites_count INT,
             statuses_count INT
            ) USING parquet
            ''')


DataFrame[]

sparksql.sql('''
    DROP TABLE user_basic_info
''')


In [7]:
a = sparksql.sql('SHOW TABLES')
a.collect()

[Row(namespace='user_data', tableName='user_basic_info', isTemporary=False)]

## Insert User Data from the tweet data

### ----------------------- ⭐️Schames⭐️ -----------------------

In [29]:
user_basic_info_schema = StructType([
    StructField("tweet_create_time", TimestampType()),
    StructField("id", LongType()),
    StructField("name", StringType()),
    StructField("screen_name", StringType()),
    StructField("verified", BooleanType()),
    StructField("created_at", TimestampType()),
    StructField("followers_count", IntegerType()),
    StructField("friends_count", IntegerType()),
    StructField("listed_count", IntegerType()),
    StructField("favourites_count", IntegerType()),
    StructField("statuses_count", IntegerType())
])

### ------------------------ ⭐️Functions⭐️ ------------------------

In [34]:
def insert_with_df(data, table_name, table_schema):
    duplicated_count = 0
    
    datetime_format = "%a %b %d %H:%M:%S %z %Y"
    tweet_create_at_datetime = datetime.strptime(data['created_at'], datetime_format)
    created_at_datetime = datetime.strptime(data['user']['created_at'], datetime_format)

    user_data_row = [(tweet_create_at_datetime, data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']["verified"], created_at_datetime, data['user']["followers_count"], data['user']["friends_count"], data['user']["listed_count"], data['user']["favourites_count"], data['user']["statuses_count"])]

    if user_exist(data['user']['id'], table_name): # if already in the table
        update_user(user_data_row, table_name, table_schema)
        duplicated_count += 1
    else: # if does not exist in table
        insert_user(user_data_row, table_name, table_schema)
    # DELETE is only supported with v2 tables.
    print(f"Warning: {duplicate_data_count} duplicate data rows were encountered. Do not forget to use the latest one.")

def user_exist(user_id, table_name) -> bool:
    result = spark.sql(f"SELECT COUNT(*) as count FROM {table_name} WHERE id = {user_id}")
    count = result.collect()[0]["count"]
    return count > 0

def update_user(data_row, table_name, table_schema):
    user_id = data_row[0][1]
    existed_ts = spark.sql(f"SELECT tweet_create_time FROM {table_name} WHERE id = {user_id}").collect()
    if len(existed_ts) > 0:
        existed_timestamp = existed_ts[0]["tweet_create_time"]
    else:
        existed_timestamp = None
    new_timestamp = data_row[0][0].replace(tzinfo=None)
    if existed_timestamp and new_timestamp and new_timestamp > existed_timestamp: # update only when timestamp is later
        print("Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.")
#         delete_user(user_id, table_name)
#         print("Deleting old user data")
        insert_user(data_row, table_name, table_schema)
#         print("Inserting new user data")
        
def insert_user(data_row, table_name, table_schema):
    user_df = spark.createDataFrame(data_row, schema=table_schema)
    user_df.write.mode("append").insertInto(table_name)
    
# def delete_user(user_id, table_name):
#     spark.sql(f"DELETE FROM {table_name} WHERE id = {user_id}")


### An example for insert directly with SQL query
query = """ \
    INSERT INTO basic (id, name, screen_name) \
    VALUES (1, 'Alice', 'alice01'), \
           (2, 'Bob', 'bob02'), \
           (3, 'Charlie', 'charlie03'), \
           (4, 'David', 'david04') \
""" \
\
spark.sql(query)

Using __DataFrames__ is more efficient then inserting data directly into a table using a SQL INSERT INTO statement, especially when dealing with __large amounts of data__.

In [ ]:
with open("/home/hx152/Project/tweet_project/Dataset/corona-out-3", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            insert_with_df(data, table_name='user_basic_info', table_schema=user_basic_info_schema)
        except json.JSONDecodeError:
            pass

Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user w

Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
23/04/21 19:16:35 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.
Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


...

Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


Oops, this user is existed. Since DELETE is only supported with v2 tables. Multiple data for this user will be stored in table.


In [8]:
spark.sql("SELECT count(*) number_rows FROM user_basic_info").show()

+-----------+
|number_rows|
+-----------+
|       9425|
+-----------+



In [9]:
spark.sql("SELECT id, name FROM user_basic_info").show()

+-------------------+---------------------------------+
|                 id|                             name|
+-------------------+---------------------------------+
|1101495976215355393|             🕉️सनातनी/देशप्रे...|
|1017594475139555329|             राष्ट्र भक्त वागस...|
|1202425207928938498|             जट्ट💪बिजेंद्र चै...|
|1202425207928938498|             जट्ट💪बिजेंद्र चै...|
| 867319391804850176|             🇮🇳राष्ट्र सर्वप...|
|1099308267132469249|             ఎప్పటికీ జనసేన కో...|
|1108021519223906307|             महेश पंडित 🚩जय श...|
|1204003466441420800|             लक्ष्मी मीणा झोंप...|
| 932051823384436736|             மன்னார்குடி கிருஷ...|
|          430915342|             जीशान उद्दीन (घर ...|
|         2148304002|             DRdhirendra sao ग...|
|          259215001|             Fight #ChineseCor...|
|         3309830041|             #ईशान#(भावी प्रधा...|
| 700640520058462208|             Rakesh Aggarwal र...|
|           54802072|南造云子（中の人岩田）@5/2ビブ...|
|1160818340